In [ ]:
import os
import torch

import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

patch_tokens = torch.load('output/tmp/patch_tokens.pt')['x_norm_patchtokens']
# print(patch_tokens.shape) # [100, 1369, 1024]
N, P, C  = patch_tokens.shape
first_frame_idx = 0

tokens_np = patch_tokens.detach().cpu().numpy().reshape(N, P * C)
pca = PCA(n_components=64, random_state=42)
patch_pca = pca.fit_transform(tokens_np)

n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)  # n_init=10避免局部最优
cluster_labels = kmeans.fit_predict(patch_pca)  # [N,]，每个样本的聚类标签
print(f"KMeans聚类完成：{n_clusters}个类别")

# --------------------------
# 4. t-SNE可视化聚类结果
# --------------------------
# t-SNE将PCA降维后的特征进一步降到2维（用于可视化）
tsne = TSNE(n_components=2, perplexity=30, random_state=42)  # perplexity根据样本量调整（5-50）
patch_tsne = tsne.fit_transform(patch_pca)  # [N, 2]

# 绘制散点图，用不同颜色表示不同聚类
plt.figure(figsize=(10, 8))
for i in range(n_clusters):
    mask = (cluster_labels == i)
    plt.scatter(patch_tsne[mask, 0], patch_tsne[mask, 1], 
                label=f"Cluster {i}", s=10, alpha=0.7)  # s=点大小，alpha=透明度
plt.scatter(patch_tsne[first_frame_idx, 0], patch_tsne[first_frame_idx, 1],
                color='red', s=100, marker='*', label="First Frame")
plt.legend()
plt.title("t-SNE Visualization of KMeans Clusters")
# plt.savefig(os.path.join(save_dir, "tsne_clusters.png"), dpi=300)
plt.show()
plt.close()

In [ ]:
first_cluster_label = cluster_labels[first_frame_idx]  # 第一帧的聚类标签

# 找到所有与第一帧标签相同的frame索引（包括第一帧自身）
same_cluster_indices = np.where(cluster_labels == first_cluster_label)[0].tolist()
print(f"第一帧索引: {first_frame_idx}，所属聚类标签: {first_cluster_label}")
print(f"与第一帧同聚类的frame索引: {same_cluster_indices}")

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

first_token = tokens_np[first_frame_idx:first_frame_idx+1]
similarities = cosine_similarity(tokens_np, first_token).squeeze()  # [S,]
# 按相似度从高到低排序，获取排序后的索引
sorted_indices = np.argsort(similarities)[::-1]  # 降序排列
sorted_similarities = similarities[sorted_indices]  # 排序后的相似度值

print(sorted_indices)
print(sorted_similarities)

In [ ]:
import os
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import glob

from utils.config_utils import load_config
from utils.dataset import load_dataset


def plot_concatenated_images(image_paths: list, 
                             indices: list, 
                             ax: plt.Axes,  # 指定绘图对象（关键：支持多次调用叠加）
                             thumb_size=(200, 200), 
                             cols=5, 
                             group_label: str = None,  # 分组标签（区分不同调用的图片组）
                             label_pos: tuple = (0.5, 1.05)):  # 分组标签位置
    """
    多次调用该函数，在指定的matplotlib Axes上绘制图片缩略图拼接网格
    （支持同一画布绘制多组图片，如“同聚类组”“高相似度组”等）
    
    Args:
        image_paths: 所有图片的路径列表（与indices对应顺序）
        indices: 需要读取的图片索引列表（如与第一帧同聚类的frame索引）
        ax: matplotlib的Axes对象（指定绘图区域，实现多次调用叠加）
        thumb_size: 缩略图尺寸 (width, height)，默认(200, 200)
        cols: 拼接图的列数，默认5列
        group_label: 该组图片的标签（如“与第一帧同聚类”“相似度Top10”），默认不显示
        label_pos: 分组标签的相对位置 (x, y)，默认在网格上方居中
    """
    # 1. 筛选需要显示的图片路径，检查有效性
    selected_paths = [image_paths[i] for i in indices]
    num_images = len(selected_paths)
    if num_images == 0:
        raise ValueError("未找到需要显示的图片索引（indices为空）")
    
    # 2. 计算网格布局（行数=图片数/列数，向上取整）
    rows = (num_images + cols - 1) // cols
    canvas_width = cols * thumb_size[0]
    canvas_height = rows * thumb_size[1]
    
    # 3. 创建空白画布（白色背景，RGB格式）
    canvas = np.ones((canvas_height, canvas_width, 3), dtype=np.uint8) * 255  # 白色背景
    
    # 4. 逐个读取图片并绘制到画布
    for idx, (img_path, original_idx) in enumerate(zip(selected_paths, indices)):
        # 计算当前缩略图在网格中的坐标（左上角）
        row = idx // cols
        col = idx % cols
        x_start = col * thumb_size[0]
        y_start = row * thumb_size[1]
        x_end = x_start + thumb_size[0]
        y_end = y_start + thumb_size[1]
        
        # 读取并处理图片（支持PNG/JPG，处理透明通道）
        try:
            with Image.open(img_path) as img:
                img_rgb = img.convert("RGB")  # 转为RGB（去除透明通道）
                img_thumb = img_rgb.resize(thumb_size, Image.LANCZOS)  # 高质量缩放
                img_np = np.array(img_thumb)  # 转为numpy数组
                canvas[y_start:y_end, x_start:x_end] = img_np  # 绘制到画布
        except Exception as e:
            # 图片读取失败：显示红色警告块
            canvas[y_start:y_end, x_start:x_end] = [255, 0, 0]  # 红色填充
            print(f"警告：图片 {img_path} 读取失败，错误：{str(e)}")
        
        # 5. 在缩略图下方标注原始索引（如“idx: 5”）
        text_x = x_start + thumb_size[0] / 2  # 水平居中
        text_y = y_end + 10  # 缩略图底部下方10px
        ax.text(text_x, text_y, f"idx: {original_idx}", 
                ha="center", va="bottom", fontsize=8, color="black",
                bbox=dict(facecolor="white", alpha=0.8, pad=1))  # 白色半透明背景
    
    # 6. 绘制分组标签（如“与第一帧同聚类的frame”）
    if group_label is not None:
        ax.text(label_pos[0], label_pos[1], group_label, 
                ha="center", va="bottom", fontsize=12, fontweight="bold",
                transform=ax.transAxes)  # 使用Axes相对坐标，避免受画布尺寸影响
    
    # 7. 在指定的Axes上显示拼接后的画布
    ax.imshow(canvas)
    ax.axis("off")  # 隐藏坐标轴（只显示图片）
    
    # 8. 调整Axes的范围（确保画布完整显示）
    ax.set_xlim(0, canvas_width)
    ax.set_ylim(canvas_height + 30, 0)  # +30预留索引文本空间，y轴翻转（匹配图片坐标系）

config = load_config('/data/xthuang/code/vggt/configs/mono/tum/fr1_desk.yaml')
dataset = load_dataset('', '', config)
image_path_list = dataset.color_paths[:100]
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(15, 12))  # 画布大小(宽, 高)
plot_concatenated_images(
    image_paths=image_path_list,
    indices=same_cluster_indices,
    ax=ax1,  # 绘制到第一个子图（上方）
    thumb_size=(180, 180),  # 缩略图尺寸
    cols=4,  # 每行3张图
    group_label="cluster Frame",  # 分组标签
    label_pos=(0.5, 1.05)  # 标签在子图上方居中
)

# --------------------------
# 4. 第二次调用：绘制“高相似度组”图片
# --------------------------
plot_concatenated_images(
    image_paths=image_path_list,
    indices=sorted_indices[:len(same_cluster_indices)],
    ax=ax2,  # 绘制到第二个子图（下方）
    thumb_size=(180, 180),
    cols=4,
    group_label="sim Frame",
    label_pos=(0.5, 1.05)
)

plt.tight_layout(pad=3.0)  # 子图间间距3.0，避免标签重叠
plt.savefig("output/tmp/multi_group_images.png", dpi=300, bbox_inches="tight")  # 保存图片
plt.show()  # 显示图片

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def visualize_attention(attn_np: np.ndarray, num_heads: int, patch_size: int = None, 
                        figsize: tuple = (12, 10), cmap: str = "viridis"):
    """
    可视化注意力热力图，展示每个query patch对key patch的关注模式
    
    参数:
        attn: 注意力权重张量，形状为 (B, H_heads, N_patches, N_patches)
              其中B=1(批量大小), H_heads为注意力头数量, N_patches为patch数量
        num_heads: 注意力头数量
        patch_size: 可选，每个patch的尺寸(用于在标题中显示)
        figsize: 图像大小
        cmap: 颜色映射方案
    """
    # 计算子图布局（尽量保持正方形）
    n_rows = int(np.ceil(np.sqrt(num_heads)))
    n_cols = int(np.ceil(num_heads / n_rows))
    
    # 创建画布
    fig, axes = plt.subplots(n_rows, n_cols, figsize=figsize)
    axes = axes.flatten()  # 展平轴数组，便于迭代
    
    # 为每个注意力头绘制热力图
    for i in range(num_heads):
        ax = axes[i]
        # 绘制热力图（颜色越深表示注意力权重越高）
        sns.heatmap(attn_np[i], ax=ax, cmap=cmap, cbar=True, 
                   vmin=0, vmax=np.max(attn_np[i]),  # 按每个头的最大值归一化显示
                   xticklabels=5, yticklabels=5)     # 每隔5个patch显示刻度
        
        # 设置标题和标签
        title = f"Head {i+1}"
        if patch_size:
            title += f" (Patch size: {patch_size}x{patch_size})"
        ax.set_title(title)
        ax.set_xlabel("Key Patch Index")
        ax.set_ylabel("Query Patch Index")
    
    # 隐藏多余的子图（当注意力头数量不是行列乘积时）
    for i in range(num_heads, n_rows * n_cols):
        axes[i].axis('off')
    
    plt.tight_layout()
    return fig

attn_list = []
for file in os.listdir('output/tmp/attn_weights'):
    attn_weight = np.load(os.path.join('output/tmp/attn_weights', file)).squeeze(0).mean(0)
    attn_list.append(attn_weight)
visualize_attention()
